# Game prototype

In [2]:
from collections.abc import Callable
from abc import ABC, abstractmethod
from typing import Any, List
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    filename="app.log",
    filemode="a",
)

In [3]:
from abc import ABC, abstractmethod
from typing import Any, List


class Sender(ABC):
    """
    Sender creates a cipher by encrypting a message.
    """

    def __init__(self, messages: List[str]) -> None:
        self.messages = messages
        self.communications = []

    @abstractmethod
    def encode(self) -> Any:
        pass


class Receiver(ABC):
    """
    Receiver tries to decrypt cipher back to message.
    """

    def __init__(self) -> None:
        self.ciphers = []
        self.communications = []

    @abstractmethod
    def decode(self, cipher: Any) -> str:
        pass


class Game(ABC):
    """
    Game logic: transmission, validation, communication.
    """

    def __init__(self, sender: Sender, receivers: List[Receiver]) -> None:
        self.sender = sender
        self.receivers = receivers
        self.points = {receiver: 0 for receiver in receivers}

    @abstractmethod
    def transmit_cipher(self, receiver: Receiver, cipher: Any, noise: Any) -> bool:
        pass

    @abstractmethod
    def verification(
        self, original_message: str, deciphered_message: str, receiver: Receiver
    ) -> int:
        pass

    @abstractmethod
    def communicate(self, receiver: Receiver, communication: str) -> bool:
        pass


In [4]:
class Player_sender(Sender):

    def __init__(self, messages):
        super().__init__(messages)
        self.logger = logging.getLogger(self.__class__.__name__)
        self.index = 0

    def encode(self):
        if self.index >= len(self.messages):
            return None

        message = self.messages[self.index]
        self.index += 1

        self.logger.debug("Encoding message (index=%d): %r", self.index-1, message)

        cipher = input(f"Encode '{message}' with emojis: ")
        return message, (cipher or None)


class Player_receiver(Receiver):

    def __init__(self):
        super().__init__()
        self.logger = logging.getLogger(self.__class__.__name__)

    def decode(self, cipher):
        self.logger.debug("Decoding cipher: %r", cipher)
        message = input(f"Decode this emoji clue '{cipher}': ")
        return message or None


class Emoji_movie_trivia(Game):

    def __init__(self, sender, receivers):
        super().__init__(sender, receivers)
        self.logger = logging.getLogger(self.__class__.__name__)

    def transmit_cipher(self, receiver, cipher, noise=None):
        self.logger.info("Transmitting cipher=%r to receiver=%r", cipher, receiver)
        receiver.ciphers.append(cipher)
        return True

    def verification(self, original_message, deciphered_message, receiver):
        self.logger.info("Verifying original=%r, deciphered=%r",
                         original_message, deciphered_message)

        if deciphered_message is None:
            print("Receiver aborted.")
            return 0

        if original_message.lower() == deciphered_message.lower():
            print("Correct!")
            self.points[receiver] += 1
            return 1

        print(f"Wrong! Correct answer was: {original_message}")
        return 0

    def communicate(self, receiver, communication):
        self.logger.debug("Sender communicates: %r", communication)
        receiver.communications.append(communication)
        return True


In [7]:
s = Player_sender(["Titanic", "Jurassic Park", "Parasite"])
r = Player_receiver()
g = Emoji_movie_trivia(s, [r])

print("Emoji Movie Trivia — START\n")

while True:

    result = s.encode()
    if result is None:
        print("No more movies!\nGame over.")
        break

    original, cipher = result
    if cipher is None:
        print("Sender aborted.")
        break

    g.transmit_cipher(r, cipher)

    guess = r.decode(cipher)
    if not guess:
        print("Receiver quit.")
        break

    g.verification(original, guess, r)

    print(f"Score: {g.points[r]}")


Emoji Movie Trivia — START

Correct!
Score: 1
Wrong! Correct answer was: Jurassic Park
Score: 1
Wrong! Correct answer was: Parasite
Score: 1
No more movies!
Game over.
